In [17]:
from random import randint
import os
import json
import settings
import pickle
from pyvi import ViTokenizer
# from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from gensim import corpora, matutils
from sklearn.metrics import classification_report
import re

## 1. Load Data

Dữ liệu sẽ được lưu vào biến để xử lý, dữ liệu đã được chia thành 2 phần data-train và data-test.

In [33]:
class FileReader(object):
    def __init__(self, filePath, encoder = None):
        self.filePath = filePath
        self.encoder = encoder if encoder != None else 'utf-16le'

    def read(self):
        with open(self.filePath, 'rb') as f:
            s = f.read()
        return s

    def content(self):
        s = self.read()
        return s.decode(self.encoder)

    def read_json(self):
        with open(self.filePath) as f:
            s = json.load(f)
        return s

    def read_stopwords(self):
        with open(self.filePath, 'r') as f:
            stopwords = set([w.strip().replace(' ', '_') for w in f.readlines()])
        return stopwords

    def load_dictionary(self):
        return corpora.Dictionary.load_from_text(self.filePath)

In [34]:
class FileStore(object):
    def __init__(self, filePath, data = None):
        self.filePath = filePath
        self.data = data

    def store_json(self):
        with open(self.filePath, 'w') as outfile:
            json.dump(self.data, outfile)

    def store_dictionary(self, dict_words):
        dictionary = corpora.Dictionary(dict_words)
        dictionary.filter_extremes(no_below=20, no_above=0.3)
        dictionary.save_as_text(self.filePath)

    def save_pickle(self,  obj):
        outfile = open(self.filePath, 'wb')
        fastPickler = pickle.Pickler(outfile, pickle.HIGHEST_PROTOCOL)
        fastPickler.fast = 1
        fastPickler.dump(obj)
        outfile.close()

In [35]:
class DataLoader(object):
    def __init__(self, dataPath):
        self.dataPath = dataPath

    def __get_files(self):
        folders = [self.dataPath + folder + '/' for folder in os.listdir(self.dataPath)]
        class_titles = os.listdir(self.dataPath)
        files = {}
        for folder, title in zip(folders, class_titles):
            files[title] = [folder + f for f in os.listdir(folder)]
        self.files = files

    def get_json(self):
        self.__get_files()
        data = []
        for topic in self.files:
            rand = randint(100, 150)
            i = 0
            for file in self.files[topic]:
                content = FileReader(filePath=file).content()
                data.append({
                    'category': topic,
                    'content': content
                })
                if i == rand:
                    break
                else:
                    i += 1
        return data

## 2. Feature Extraction

Sau khi đã có tập dữ liệu, tiến hành một số bước lựa chọn thuộc tính đầu vào cho bài toán phân lớp.

- Words segmentation `def segmentation(self):`
- Remove Stopwords `NLP(object):`
- Xây dựng từ điển các từ `FeatureExtraction(object):`
- Khởi tạo vector thuộc tính với Bag of Word `FeatureExtraction(object):`


In [36]:
class NLP(object):
    def __init__(self, text = None):
        self.text = text
        self.__set_stopwords()

    def __set_stopwords(self):
        self.stopwords = FileReader(settings.STOP_WORDS).read_stopwords()

    def segmentation(self):
        return ViTokenizer.tokenize(self.text)

    def split_words(self):
        text = self.segmentation()
        try:
            return [x.strip(settings.SPECIAL_CHARACTER).lower() for x in text.split()]
        except TypeError:
            return []
            

    def get_words_feature(self):
        split_words = self.split_words()
        return [word for word in split_words if word.encode('utf-8') not in self.stopwords]

In [37]:
class FeatureExtraction(object):
    def __init__(self, data):
        self.data = data

    def __build_dictionary(self):
        print ('Building dictionary')
        dict_words = []
        i = 0
        for text in self.data:
            i += 1
            print ("Step {} / {}".format(i, len(self.data)))
            words = NLP(text = text['content']).get_words_feature()
            dict_words.append(words)
        FileStore(filePath=settings.DICTIONARY_PATH).store_dictionary(dict_words)

    def __load_dictionary(self):
        if os.path.exists(settings.DICTIONARY_PATH) == False:
            self.__build_dictionary()
        self.dictionary = FileReader(settings.DICTIONARY_PATH).load_dictionary()

    def __build_dataset(self):
        self.features = []
        self.labels = []
        i = 0
        for d in self.data:
            i += 1
            print( "Step {} / {}".format(i, len(self.data)))
            self.features.append(self.get_dense(d['content']))
            self.labels.append(d['category'])
    
    def get_dense(self, text):
        self.__load_dictionary()
        # row = re.sub(r"[\.,\?]+$-", "", text)
        # # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
        # row = row.replace(",", " ").replace(".", " ") \
        #     .replace(";", " ").replace("“", " ") \
        #     .replace(":", " ").replace("”", " ") \
        #     .replace('"', " ").replace("'", " ") \
        #     .replace("!", " ").replace("?", " ") \
        #     .replace("-", " ").replace("?", " ") \
        #     .replace("(", " ").replace(")", " ")
        # text = re.sub(r" {2, }", " ", row).strip().lower()
        words = NLP(text).get_words_feature()
        # Bag of words
        vec = self.dictionary.doc2bow(words)
        dense = list(matutils.corpus2dense([vec], num_terms=len(self.dictionary)).T[0])
        return dense

    def get_data_and_label(self):
        self.__build_dataset()
        return self.features, self.labels

## 3. Phân lớp văn bản Naive Bayes

Sau khi đã có được vector thuộc tính sử dụng phương pháp **Bag of Word** sẽ tiến hành phân loại văn bản.


In [38]:
class Classifier(object):
    def __init__(self, features_train = None, labels_train = None, features_test = None, labels_test = None,  estimator = GaussianNB(priors=None)):
        self.features_train = features_train
        self.features_test = features_test
        self.labels_train = labels_train
        self.labels_test = labels_test
        self.estimator = estimator

    def training(self):
        self.estimator.fit(self.features_train, self.labels_train)
        self.__training_result()

    def save_model(self, filePath):
        FileStore(filePath=filePath).save_pickle(obj=est)

    def __training_result(self):
        y_true, y_pred = self.labels_test, self.estimator.predict(self.features_test)
        print(classification_report(y_true, y_pred))

In [39]:
if __name__ == '__main__':
    json_train = DataLoader(dataPath=settings.DATA_TRAIN_PATH).get_json()
    FileStore(filePath=settings.DATA_TRAIN_JSON, data=json_train).store_json()
    json_test = DataLoader(dataPath=settings.DATA_TEST_PATH).get_json()
    FileStore(filePath=settings.DATA_TEST_JSON, data=json_test).store_json()
    train_loader = FileReader(filePath=settings.DATA_TRAIN_JSON)
    test_loader = FileReader(filePath=settings.DATA_TEST_JSON)
    data_train = train_loader.read_json()
    data_test = test_loader.read_json()

    features_train, labels_train = FeatureExtraction(data=data_train).get_data_and_label()
    features_test, labels_test = FeatureExtraction(data=data_test).get_data_and_label()

    est = Classifier(features_train=features_train, features_test=features_test, labels_train=labels_train, labels_test=labels_test)
    est.training()
    est.save_model(filePath='trained_model/gaussiannb_nb_model.pk')
    print ('Finish processing.')

Step 1 / 1301
Step 2 / 1301
Step 3 / 1301
Step 4 / 1301
Step 5 / 1301
Step 6 / 1301
Step 7 / 1301
Step 8 / 1301
Step 9 / 1301
Step 10 / 1301
Step 11 / 1301
Step 12 / 1301
Step 13 / 1301
Step 14 / 1301
Step 15 / 1301
Step 16 / 1301
Step 17 / 1301
Step 18 / 1301
Step 19 / 1301
Step 20 / 1301
Step 21 / 1301
Step 22 / 1301
Step 23 / 1301
Step 24 / 1301
Step 25 / 1301
Step 26 / 1301
Step 27 / 1301
Step 28 / 1301
Step 29 / 1301
Step 30 / 1301
Step 31 / 1301
Step 32 / 1301
Step 33 / 1301
Step 34 / 1301
Step 35 / 1301
Step 36 / 1301
Step 37 / 1301
Step 38 / 1301
Step 39 / 1301
Step 40 / 1301
Step 41 / 1301
Step 42 / 1301
Step 43 / 1301
Step 44 / 1301
Step 45 / 1301
Step 46 / 1301
Step 47 / 1301
Step 48 / 1301
Step 49 / 1301
Step 50 / 1301
Step 51 / 1301
Step 52 / 1301
Step 53 / 1301
Step 54 / 1301
Step 55 / 1301
Step 56 / 1301
Step 57 / 1301
Step 58 / 1301
Step 59 / 1301
Step 60 / 1301
Step 61 / 1301
Step 62 / 1301
Step 63 / 1301
Step 64 / 1301
Step 65 / 1301
Step 66 / 1301
Step 67 / 1301
Step

In [40]:
features_train, labels_train = FeatureExtraction(data=data_train).get_data_and_label()
features_test, labels_test = FeatureExtraction(data=data_test).get_data_and_label()


Step 1 / 1301
Step 2 / 1301
Step 3 / 1301
Step 4 / 1301
Step 5 / 1301
Step 6 / 1301
Step 7 / 1301
Step 8 / 1301
Step 9 / 1301
Step 10 / 1301
Step 11 / 1301
Step 12 / 1301
Step 13 / 1301
Step 14 / 1301
Step 15 / 1301
Step 16 / 1301
Step 17 / 1301
Step 18 / 1301
Step 19 / 1301
Step 20 / 1301
Step 21 / 1301
Step 22 / 1301
Step 23 / 1301
Step 24 / 1301
Step 25 / 1301
Step 26 / 1301
Step 27 / 1301
Step 28 / 1301
Step 29 / 1301
Step 30 / 1301
Step 31 / 1301
Step 32 / 1301
Step 33 / 1301
Step 34 / 1301
Step 35 / 1301
Step 36 / 1301
Step 37 / 1301
Step 38 / 1301
Step 39 / 1301
Step 40 / 1301
Step 41 / 1301
Step 42 / 1301
Step 43 / 1301
Step 44 / 1301
Step 45 / 1301
Step 46 / 1301
Step 47 / 1301
Step 48 / 1301
Step 49 / 1301
Step 50 / 1301
Step 51 / 1301
Step 52 / 1301
Step 53 / 1301
Step 54 / 1301
Step 55 / 1301
Step 56 / 1301
Step 57 / 1301
Step 58 / 1301
Step 59 / 1301
Step 60 / 1301
Step 61 / 1301
Step 62 / 1301
Step 63 / 1301
Step 64 / 1301
Step 65 / 1301
Step 66 / 1301
Step 67 / 1301
Step

In [26]:
print(data_test[0]['content'])
row = re.sub(r"[\.,\?]+$-", "", data_test[0]['content'])
        # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
row = row.replace(",", " ").replace(".", " ") \
            .replace(";", " ").replace("“", " ") \
            .replace(":", " ").replace("”", " ") \
            .replace('"', " ").replace("'", " ") \
            .replace("!", " ").replace("?", " ") \
            .replace("-", " ").replace("?", " ") \
            .replace("(", " ").replace(")", " ")
text = re.sub(r" {2, }", " ", row).strip().lower()
print(text)
words = NLP(text).get_words_feature()
print(words)

Voi rừng phá hoại hoa màu ở Đăk Lăk
Ông Ysum Ê Ban, Phó chủ tịch UNND xã Ea Rốc, huyện Ea Súp, cho biết đêm mùng 3 rạng sáng ngày 4/8, một đàn voi rừng khoảng 10 con, có cả voi con, đã xuất hiện tại tiểu khu 204 thuộc lâm phần do lâm trường Rừng xanh quản lý, cách trung tâm xã 10 km. 
Chỉ trong 1 đêm, đàn voi đã phá nát 5 ha bắp lai trong tổng diện tích 30 ha của nhân dân xã Ea Rốc đang trong thời kỳ sắp thu hoạch, đàn voi có vẻ đói khát và hung dữ. 
Trước đó, vào các ngày 31/7 và 1/8, tại tiểu khu 173, 190 thuộc địa phận lâm trường Ea Hmơ và Y lốp, đàn voi trên đã phá nát gần 30 ha bắp và đậu đỗ các loại đang trong thời kỳ thu hoạch. Đàn voi không chỉ xuất hiện ban đêm mà cả ban ngày, gây lo sợ trong nhân dân và ảnh hưởng rất lớn đến sản xuất.
Mặc dù lực lượng dân quân của các xã và tự vệ lâm trường đã dùng các biện pháp xua đuổi song đàn voi xem ra chẳng sợ. Hiện nay, cấp uỷ quyền huyện Ea Súp cùng các địa phương và cơ quan chức năng đang chỉ đạo nhân dân tiếp tục canh gác, dùng các 

In [27]:
est = Classifier(features_train=features_train, features_test=features_test, labels_train=labels_train, labels_test=labels_test)
est.training()


                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.44      0.72      0.55       143
        Doi song       0.68      0.38      0.49       133
        Khoa hoc       0.52      0.62      0.56       104
      Kinh doanh       0.89      0.63      0.74       132
       Phap luat       0.82      0.67      0.74       129
        Suc khoe       0.80      0.80      0.80       124
        The gioi       0.85      0.81      0.82       149
        The thao       0.95      0.92      0.93       123
         Van hoa       0.64      0.84      0.73       121
         Vi tinh       0.89      0.80      0.84       131

        accuracy                           0.72      1289
       macro avg       0.75      0.72      0.72      1289
    weighted avg       0.75      0.72      0.72      1289



In [41]:
est = Classifier(features_train=features_train, features_test=features_test, labels_train=labels_train, labels_test=labels_test)
est.training()

                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.40      0.70      0.51       124
        Doi song       0.62      0.27      0.37       142
        Khoa hoc       0.58      0.66      0.62       110
      Kinh doanh       0.87      0.57      0.69       134
       Phap luat       0.86      0.63      0.72       126
        Suc khoe       0.87      0.78      0.82       125
        The gioi       0.79      0.82      0.80       145
        The thao       0.96      0.91      0.94       128
         Van hoa       0.56      0.88      0.69       121
         Vi tinh       0.87      0.84      0.85       147

        accuracy                           0.70      1302
       macro avg       0.74      0.71      0.70      1302
    weighted avg       0.74      0.70      0.70      1302

